In [1]:
import sys
import pickle
import traceback
from pprint import pprint
import numpy as np
import pandas as pd
from gensim import corpora,models

from scipy.sparse import *
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Model
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Convolution1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.layers import Activation

import jieba
from utils.data_utils import clean_str
from utils.data_utils import build_vocab
from utils.data_utils import get_tokens

Using TensorFlow backend.


In [46]:
df_dataset = pd.read_csv('./data/training-inspur.csv', encoding='utf-8')

COMMCONTENT_SEG = []

for sent in df_dataset['COMMCONTENT']:

    # Extract Sentence
    sent = str(sent).strip()

    # 去除标点符号会导致 Accuracy 降低
    # sent = clean_str(sent)

    seg_list = jieba.cut(sent, cut_all=False)

    seg_list = [i for i in seg_list if i!=' ']
    
    COMMCONTENT_SEG.append(" ".join(seg_list))
    
df_dataset['COMMCONTENT_SEG'] = pd.DataFrame(COMMCONTENT_SEG,columns=['COMMCONTENT_SEG'])
df_dataset = df_dataset[df_dataset['COMMCONTENT_SEG']!=""]
df_dataset = df_dataset.reset_index()

In [22]:
COMMCONTENT_SEG

['普通 公园 一个 只是 多 了 几个 泉 而已 ， 人不多 ， 适合 老人 孩子 闲逛 ， 买票 的话 还是 贵 了 ， 人家 说 6.30 之 前进 园 不用 花钱',
 '跟 儿子 在 里面 玩 了 一天 ， 非常 好 ！ 跟 儿子 在 里面 玩 了 一天 ， 非常 好 ！ 真的 很 不错 哦 ， 有空 还要 去',
 '这 已经 是 第五次 来 这里 玩 了 。 每次 孩子 都 很 喜欢 ， 不 愿意 从水里 出来 。 有 机会 还会 再 来 。 还有 比 我 更 忠诚 的 客户 吗 ？ 哈哈',
 '当天 在 携程 上定 的 票 ， 打 温泉 度假村 咨询电话 和 携程 客服 都 说 次日 生效 ， 但 到 酒店 后 ， 票能 用 。 请客 服 人员 了解 清楚 再 回答 咨询 问题 。 不然 听信 ， 就 得 中途 掉头 回家 了 。',
 '烟台 历史 的 一部分 ， 非常 值得 推荐 去 看看 ！ 海边 景色 也 很漂亮 ！',
 '周末 看看 动物 亲近 亲近 大自然 挺 好 的 ， 媳妇儿 还 跟 猴子 拍照 ， 猴子 满身 爬 ， 挺好玩 ， 如果 动物 再 多点 就 好 了 ， 门票 小贵',
 '五四 广场 青岛 旅游景点 必 打卡 又 一 地点 ， 标志性 红包 建筑 雕塑 矗立 在 市政府 对面 亦 是 海边 ， 还是 有点 意思 的 ， 旁边 有 游船 可 出海 游玩',
 '五四 广场 坐落 在 山东省 青岛市 市南区 ， 在 海边 ， 那个 红红的 火炬 就是 五四 广场 的 标志 。 广场 上 有 大片 绿地 ， 广场 上 游玩 的 人 挺 多 的 ， 聚集 火车站 不太远 。',
 '环境 好 ， 景色 美 ， 值得 游览 ， 感觉 很 好',
 '青岛市 中心广场 ， 纪念 五四运动 而 命名 ， 风景 独特',
 '岱庙 历史 已经 达 千年 以上 ， 岱庙 是 古代 皇帝 来 泰安 的 行宫 ， 历朝历代 留下 的 碑刻 现在 已经 是 国宝 了 ， 李斯 小篆 、 张迁碑 、 双束 碑 等 都 在 国宝 档案 中 介绍 过 ， 乾隆皇帝 赐给 泰山 的 三件 宝贝 ： 沉香 狮子 、 温凉玉圭 、 黄釉 葫芦 瓶 就 放在 岱庙 ， 汉柏 、 唐槐 等 古树 苍天 ， 门票 且 非常 便宜 ， 来 泰安 必到 之处 

In [47]:
COMMCONTENT_SEG[1].split(' ')

['跟',
 '儿子',
 '在',
 '里面',
 '玩',
 '了',
 '一天',
 '非常',
 '好',
 '跟',
 '儿子',
 '在',
 '里面',
 '玩',
 '了',
 '一天',
 '非常',
 '好',
 '真的',
 '很',
 '不错',
 '哦',
 '有空',
 '还要',
 '去']

In [49]:
df_dataset.head()

,index,ROWKEY,COMMCONTENT,COMMLEVEL,COMMCONTENT_SEG
0,0,1080003,普通公园一个只是多了几个泉而已，人不多，适合老人孩子闲逛，买票的话还是贵了，人家说6.30之...,1,普通 公园 一个 只是 多 了 几个 泉 而已 人不多 适合 老人 孩子 闲逛 买票 的话 ...
1,1,1080004,跟儿子在里面玩了一天，非常好！跟儿子在里面玩了一天，非常好！真的很不错哦，有空还要去,1,跟 儿子 在 里面 玩 了 一天 非常 好 跟 儿子 在 里面 玩 了 一天 非常 好 真的...
2,2,1080005,这已经是第五次来这里玩了。每次孩子都很喜欢，不愿意从水里出来。有机会还会再来。还有比我更忠诚...,1,这 已经 是 第五次 来 这里 玩 了 每次 孩子 都 很 喜欢 不 愿意 从水里 出来 有...
3,3,1080006,当天在携程上定的票，打温泉度假村咨询电话和携程客服都说次日生效，但到酒店后，票能用。请客服人...,1,当天 在 携程 上定 的 票 打 温泉 度假村 咨询电话 和 携程 客服 都 说 次日 生效...
4,4,1080007,烟台历史的一部分，非常值得推荐去看看！海边景色也很漂亮！,1,烟台 历史 的 一部分 非常 值得 推荐 去 看看 海边 景色 也 很漂亮


In [50]:
BASE_DIR = '/Users/tsw/ScenicSpotReviews'

W2V_DIR = BASE_DIR + '/embeddings/'

TEXT_DATA_DIR = BASE_DIR + '/data/'

MAX_SEQUENCE_LENGTH = 100

MAX_NUM_WORDS = 33950

MAX_NB_WORDS = 30000

EMBEDDING_DIM = 300

VALIDATION_SPLIT = 0.2

BATCH_SIZE = 32

In [51]:
vocab,vocab_freqs = build_vocab(df_dataset['COMMCONTENT_SEG'])

In [52]:
vocab_size = min(MAX_NB_WORDS, len(vocab_freqs)) + 2
word2index = {x[0]: i+2 for i, x in enumerate(vocab_freqs.most_common(MAX_NB_WORDS))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

In [45]:
word_index

{'，': 1,
 '的': 2,
 '。': 3,
 '了': 4,
 '是': 5,
 '！': 6,
 '很': 7,
 '去': 8,
 '也': 9,
 '不': 10,
 '都': 11,
 '在': 12,
 '就': 13,
 '有': 14,
 '还': 15,
 '没有': 16,
 '好': 17,
 '我': 18,
 '不错': 19,
 '可以': 20,
 '人': 21,
 '就是': 22,
 '景区': 23,
 '、': 24,
 '一个': 25,
 '到': 26,
 '没': 27,
 '地方': 28,
 '景点': 29,
 '和': 30,
 '玩': 31,
 '说': 32,
 '感觉': 33,
 '多': 34,
 '门票': 35,
 '还是': 36,
 '看': 37,
 '里面': 38,
 '很多': 39,
 '要': 40,
 '我们': 41,
 '非常': 42,
 '孩子': 43,
 '比较': 44,
 '吧': 45,
 '来': 46,
 '值得': 47,
 '…': 48,
 '一般': 49,
 '什么': 50,
 '青岛': 51,
 '但是': 52,
 '给': 53,
 '太': 54,
 '挺': 55,
 '这里': 56,
 '这个': 57,
 '个': 58,
 '上': 59,
 '时候': 60,
 '不是': 61,
 '大': 62,
 '走': 63,
 '你': 64,
 '票': 65,
 '景色': 66,
 '元': 67,
 '还有': 68,
 '着': 69,
 '买': 70,
 '？': 71,
 '特别': 72,
 '小时': 73,
 '时间': 74,
 '有点': 75,
 '啊': 76,
 '没什么': 77,
 '但': 78,
 '项目': 79,
 '让': 80,
 '进去': 81,
 '再': 82,
 '这': 83,
 '等': 84,
 '带': 85,
 '方便': 86,
 '真的': 87,
 '又': 88,
 '不过': 89,
 '小': 90,
 '建议': 91,
 '而且': 92,
 '能': 93,
 '排队': 94,
 '服务': 95,
 '看看': 96,
 '才':

In [53]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

tokenizer.fit_on_texts(list(df_dataset['COMMCONTENT_SEG']))

dataset_sequences = tokenizer.texts_to_sequences(list(df_dataset['COMMCONTENT_SEG']))
word_index = tokenizer.word_index

In [54]:
dataset_sequences

[[558,
  106,
  21,
  162,
  30,
  2,
  204,
  567,
  539,
  650,
  99,
  286,
  39,
  5131,
  179,
  193,
  32,
  104,
  2,
  709,
  28,
  462,
  260,
  420,
  4415,
  649,
  338,
  679],
 [153,
  732,
  9,
  34,
  27,
  2,
  268,
  38,
  14,
  153,
  732,
  9,
  34,
  27,
  2,
  268,
  38,
  14,
  81,
  4,
  16,
  347,
  2039,
  247,
  5],
 [77,
  170,
  3,
  16664,
  42,
  51,
  27,
  2,
  1134,
  39,
  8,
  4,
  97,
  7,
  1556,
  11843,
  182,
  11,
  447,
  540,
  76,
  42,
  63,
  133,
  15,
  200,
  11844,
  1,
  1475,
  287,
  619],
 [572,
  9,
  189,
  4162,
  1,
  60,
  255,
  149,
  2337,
  9464,
  26,
  189,
  132,
  8,
  28,
  9465,
  7965,
  72,
  22,
  356,
  123,
  16665,
  159,
  16666,
  4737,
  328,
  395,
  733,
  76,
  2399,
  1695,
  373,
  834,
  11845,
  10,
  131,
  2089,
  5622,
  1856,
  2],
 [530, 151, 1, 1578, 38, 43, 112, 5, 90, 206, 61, 6, 344],
 [464,
  90,
  128,
  5132,
  5132,
  1058,
  50,
  14,
  1,
  16667,
  12,
  153,
  2040,
  367,
  2040,
  79

In [55]:
dataset_sequences = tokenizer.texts_to_matrix(list(df_dataset['COMMCONTENT_SEG']), mode='binary')

In [56]:
train_X,valid_X,train_y,valid_y =train_test_split(dataset_sequences, df_dataset['COMMLEVEL'], test_size=0.2)

In [33]:
train_X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
# keras.preprocessing.text.one_hot(text, len(vocab), filters=',!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ', split=' ')

In [57]:
for i in valid_X[10]:
    if(i!=0):
        print(i)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [58]:
print('Found %s unique tokens.' % len(word_index))

Found 33827 unique tokens.


In [59]:
model = Sequential()
# FC Layer
model.add(Dense(512, input_shape=(MAX_NB_WORDS,), activation='relu'))
# dropout
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               15360512  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 1539      
Total params: 15,362,051
Trainable params: 15,362,051
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
batch_size=256
epochs = 3

In [63]:
model.fit(x=train_X, y=to_categorical(train_y-1, num_classes=None), 
                    validation_data=(valid_X, to_categorical(valid_y-1, num_classes=None)[:]), 
                    batch_size=batch_size, 
                    #callbacks=[checkpoint],
#                     callbacks=[history],
                    epochs=epochs,
                    verbose=1
         )

Train on 15995 samples, validate on 3999 samples
Epoch 1/3
15995/15995 [==============================] - 28s 2ms/step - loss: 0.8306 - acc: 0.6353 - val_loss: 0.7340 - val_acc: 0.6774
Epoch 2/3
15995/15995 [==============================] - 27s 2ms/step - loss: 0.5569 - acc: 0.7783 - val_loss: 0.7437 - val_acc: 0.6782
Epoch 3/3
15995/15995 [==============================] - 27s 2ms/step - loss: 0.3989 - acc: 0.8570 - val_loss: 0.8014 - val_acc: 0.6729
